langchain Baby AGI approach
===========================

While doing research I found a python notebook in the langchain github repositories that shows a more advanced way of using agents. This notebook approaches using TODO lists and other techniques copied from this [notebook](https://github.com/langchain-ai/langchain/blob/master/cookbook/baby_agi_with_agent.ipynb)


Unfortunately Langchains implementation of BabyAGI no longer works as it is using older versions of the langchain_experimental libraries. I do not want to use old versions as they might be missing important featrues. However, the implementation seems simple enough for me to replicate from scratch.

Setting up
==========

NOTE: All of this is outdated, I have since learned that this is not accurate to how this works.

Langchain's babyAGI implementation uses three chains: Task creation, Task prioritization, and execution. The first two being simple chains, while the last is an agent with tools.

In the original implementation, the executor chain is the main 'brain' of the system, while the todo-list and prioritization chains are used as tools that the executor can call upon.

General imports
---------------


In [286]:
import os
import openai
from dotenv import load_dotenv

from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.schema.language_model import BaseLanguageModel
from langchain.agents import Tool
from langchain.agents import ZeroShotAgent
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent
from langchain.tools import ShellTool
from langchain.python import PythonREPL

# Reomve line if needs to be set to false
os.environ["VERBOSE"] = "true"
load_dotenv()

True

Task creation chain
-------------------
(credit: This code is almost entirely copied from the [original langchain implementation](https://github.com/langchain-ai/langchain/blob/master/libs/experimental/langchain_experimental/autonomous_agents/baby_agi/task_creation.py))

In [287]:
class TaskCreationChain(LLMChain):
    """Chain generating tasks."""

    @classmethod
    def from_llm(cls, llm: BaseLanguageModel) -> LLMChain:
        """
            Get the response parser.

            Args:
                - cls: A langchain chain
                - llm: A wrapper around the language model being used
        """
        task_creation_template = (
            "You are an task creation AI that uses the result of an execution agent"
            " to create new tasks with the following objective: {objective},"
            " The last completed task has the result: {result}."
            " This result was based on this task description: {task_description}."
            " These are incomplete tasks: {incomplete_tasks}."
            " Based on the result, create new tasks to be completed"
            " by the AI system that do not overlap with incomplete tasks."
            " Return the tasks as an array."
        )
        prompt = PromptTemplate(
            template=task_creation_template,
            input_variables=[
                "result",
                "task_description",
                "incomplete_tasks",
                "objective",
            ],
        )
        return cls(prompt=prompt, llm=llm, verbose=bool(os.getenv("VERBOSE", False)))

Task prioritization chain
-------------------------

(credit: This code is almost entirely copied from the [original langchain implementation](https://github.com/langchain-ai/langchain/blob/master/libs/experimental/langchain_experimental/autonomous_agents/baby_agi/task_prioritization.py))

In [288]:

class TaskPrioritizationChain(LLMChain):
    """Chain to prioritize tasks."""

    @classmethod
    def from_llm(cls, llm: BaseLanguageModel) -> LLMChain:
        """
            Get the response parser.

            Args:
                - cls: A langchain chain
                - llm: A wrapper around the language model being used
        """
        task_prioritization_template = (
            "You are a task prioritization AI tasked with cleaning the formatting of "
            "and reprioritizing the following tasks: {task_names}."
            " Consider the ultimate objective of your team: {objective}."
            " Do not remove any tasks. Return the result as a numbered list, like:"
            " #. First task"
            " #. Second task"
            " Start the task list with number {next_task_id}."
        )
        prompt = PromptTemplate(
            template=task_prioritization_template,
            input_variables=["task_names", "next_task_id", "objective"],
        )
        return cls(prompt=prompt, llm=llm, verbose=bool(os.getenv("VERBOSE", False)))

Task execution chain
--------------------

(credit: This code is in part copied from the [original langchain implementation](https://github.com/langchain-ai/langchain/blob/master/libs/experimental/langchain_experimental/autonomous_agents/baby_agi/task_execution.py))

In [289]:
class TaskExecutionChain(LLMChain):
    """Chain to execute tasks."""

    @classmethod
    def from_llm(cls, llm: BaseLanguageModel) -> LLMChain:
        """
            Get the response parser.

            Args:
                - cls: A langchain chain
                - llm: A wrapper around the language model being used
        """
        execution_template = (
            "You are an AI who performs one task based on the following objective: "
            "{objective}."
            "Take into account these previously completed tasks: {context}."
            " Your task: {task}. Response:"
        )
        prompt = PromptTemplate(
            template=execution_template,
            input_variables=["objective", "context", "task"],
        )
        return cls(prompt=prompt, llm=llm, verbose=bool(os.getenv("VERBOSE", False)))

NOTE: For the time being im going to start implementing this without any long-term memory. Later I think it might be useful to create an entity based memory either as a langchain object or a query tool that the agent can use to look up information about an entity.

Tools
-----

In [290]:
shell_tool = ShellTool()
python_tool = PythonREPLTool()

In [291]:
# Create a chain thats only goal is to set up the initial todo-list
todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a todo list for a given objective. Come up with a todo list for this objective: {objective}. Keep it simple, the fewer tasks the better. Do not install anything!"
)
todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)
todo_tool = Tool(
        name="TODO",
        func=todo_chain.run,
        description="useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!",
    )

Output parsers
--------------
Output parsers are functions that parse the output from the language model into something usable in python.

#### TODO list parser

Parse the todo list.

In [292]:
import re
def extract_todolist(text):
    pattern = r'\n\d+\.\s(.*?)(?=\n|$)'
    matches = re.findall(pattern, text, re.DOTALL)
    return matches


Implementation
--------------

In [293]:
#objective = "Scan the current network, find out everything you can about each device and attempt to guess what the device is."
#objective = "Create a simple website that just says 'hello world' and host it on the local network using php -S."
#objective = "Scan the current network, tell me as much as you can on each device:w"
objective = "Scan the current network."

In [294]:
prefix = """You are an AI who performs a single task based on the following objective: '{objective}'. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=['objective', 'context', 'task']
)

In [295]:
llm = ChatOpenAI(temperature=0.5,)# model="gpt-4")
llm_chain = LLMChain(llm=llm, prompt=prompt)

agent_todo = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=[todo_tool.name])

agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=[tool.name for tool in [python_tool, shell_tool]])

agent_executor_w_todo = AgentExecutor.from_agent_and_tools(
    agent=agent_todo, tools=[todo_tool], verbose=True
)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=[python_tool, shell_tool], verbose=True
)

`agent_scratchpad` should be a variable in prompt.input_variables. Did not find it, so adding it at the end.


In [296]:

context = ""
task_list = ["create a todo list"]
output = agent_executor_w_todo.run(prompt=prompt, task=task_list[0], objective=objective, context=context)
completed_tasks = [task_list[0]]
task_list = extract_todolist(output)
context += f"Question: {completed_tasks[-1]}\nFinal Answer: {output}\n"


while len(task_list) > 0:
    print("-------------------------- new task -----------------------------")
    output = agent_executor.run(prompt=prompt, task=task_list[0], objective=objective, context=context)
    completed_tasks.append(task_list.pop(0))
    context += f"Question: {completed_tasks[-1]}\nFinal Answer: {output}\n"
    print("\n----------------------------- context --------------------\n", context)






> Entering new AgentExecutor chain...
Thought: To create a todo list, I need to generate a list of tasks based on the given objective.
Action: TODO
Action Input: 'Scan the current network.'
Observation: 

1. Gather information about the current network (IP addresses, devices, etc.)
2. Create a network diagram of the current network
3. Perform a port scan of the current network
4. Perform a vulnerability scan of the current network
5. Analyze the results of the scans and identify any potential security risks
Thought:I have generated a list of tasks based on the given objective.

Final Answer: 
1. Gather information about the current network (IP addresses, devices, etc.)
2. Create a network diagram of the current network
3. Perform a port scan of the current network
4. Perform a vulnerability scan of the current network
5. Analyze the results of the scans and identify any potential security risks

> Finished chain.
-------------------------- new task -----------------------------


> E

Python REPL can execute arbitrary code. Use with caution.


I can use Python to gather information about the current network and create a network diagram. 
Action: Python_REPL
Action Input: 
```
import networkx as nx
import matplotlib.pyplot as plt

# Gather information about the current network
# TODO: Write code to gather network information

# Create a network diagram
G = nx.Graph()
# TODO: Add nodes and edges to the network diagram based on the gathered network information

# Draw the network diagram
nx.draw(G, with_labels=True)
plt.show()
```
Observation: ModuleNotFoundError("No module named 'networkx'")
Thought:I need to install the `networkx` module in order to create a network diagram using Python.
Action: terminal
Action Input: `pip install networkx`

/home/antone/Music/The-Autonomous-R.A.T.-Machine/research/basic-hacker-agent/.venv/lib/python3.11/site-packages/langchain/tools/shell/tool.py:32: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(



Observation: 
[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip
/bin/sh: line 1: Collecting: command not found

Thought:The `pip` command is not recognized in the terminal. I need to check if `pip` is installed on the system.
Action: terminal
Action Input: `pip --version`
Observation: ERROR: unknown command "23.2.1"

Thought:The `pip` command is not recognized in the terminal, which means `pip` is not installed on the system.
Action: terminal
Action Input: `sudo apt-get install python3-pip`
Observation: sudo: a terminal is required to read the password; either use the -S option to read from standard input or configure an askpass helper
sudo: a password is required

Thought:I need root access to install `pip`. I should use the `sudo` command to run the installation with root privileges.
Action: terminal
Action Input: `sudo apt-get install python3-pip`
Observation: sudo: a terminal is required to read the password; either use t

OutputParserException: Could not parse LLM output: `I need root access to install `pip`. I should use the `sudo` command to run the installation with root privileges. However, as an AI, I don't have the ability to input a password for `sudo`. I will need to ask the user to provide the password or run the command on a system where I have the necessary permissions.`